In [ ]:
import pyspark.sql.functions as F

In [ ]:
%run ../../SSB_Spark_tools/utils/listcode_functions.py

In [ ]:
%run ../../SSB_Spark_tools/data_processing/Utils.py

In [ ]:
# Tar tverrsnitt av skattemeldinger og pakker de ut
tverrsnitt = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding')
test_dict = unpack_parquet(tverrsnitt, rootdf=True, rootvar=['personidentifikator'])

test_dict['loennOgTilsvarendeYtelser_koblet'] = test_dict['loennOgTilsvarendeYtelser_samledeYtelserFraArbeidsgiverPerBehandlingsart']\
            .join(test_dict['loennOgTilsvarendeYtelser'].select('loennOgTilsvarendeYtelser_id', 'overfoertFraBarn'),\
            'loennOgTilsvarendeYtelser_id', how='left')

In [ ]:
#Eksempel som viser hvordan hente inn kodeliste og kjøre sjekk med listcode_check funksjonen
ba_kodeliste_df = spark.read.path('/kilde/ske/skatt/person/2019/kodelister/behandlingsart')

#Gjør om kodeliste til python liste
ba_kodeliste = []
for row in ba_kodeliste_df.rdd.collect():
    ba_kodeliste.append(row['kode_behandlingsart'])

#Kjører sjekk
sjekk, sjekkdf = listcode_check(test_dict['loennOgTilsvarendeYtelser_koblet'], 'behandlingsart', ba_kodeliste)    

#Skriver ut dataframe hvis det er noen som ikke er i kodelisten
if (sjekk == False):
    ikke_i_kodeliste = sjekkdf.filter(F.col('i_kodeliste')==False)
    ikke_i_kodeliste.show()

In [ ]:
#Test som ovenfor men nå med manglende verdier i liste (som er generert selv) for å kunne se output

#Lager egen kodeliste som det sjekkes mot
negtest = ['MAT_OVERTID', 'TFRI_KOST_TJENESTE']

#Kjører sjekk
sjekk, sjekkdf = listcode_check(test_dict['loennOgTilsvarendeYtelser_koblet'], 'behandlingsart', negtest)

#Skriver ut dataframe hvis det er noen som ikke er i kodelisten
if (sjekk == False):
    ikke_i_kodeliste = sjekkdf.filter(F.col('i_kodeliste')==False)
    ikke_i_kodeliste.show()

In [ ]:
#Eksempel på bruk av listcode_lookup funksjonen
test = listcode_lookup(test_dict['loennOgTilsvarendeYtelser_koblet'], 'behandlingsart', ba_kodeliste_df, ['kode_behandlingsart', 'kodetekst_behandlingsart'])    
test.toPandas()